In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

data_path = Path('/kaggle/input/h-and-m-personalized-fashion-recommendations/')

In [ ]:
df = pd.read_csv(
    data_path / 'transactions_train.csv',
    # set dtype or pandas will drop the leading '0' and convert to int
    dtype={'article_id': str} 
)

submission = pd.read_csv(data_path / 'sample_submission.csv')

In [ ]:
print(df.shape)
df.head()

In [ ]:
df['t_dat'] = pd.to_datetime(df['t_dat'])
df['t_dat'].head()

In [ ]:
df_3w = df[df['t_dat'] >= pd.to_datetime('2020-08-31')].copy()
df_2w = df[df['t_dat'] >= pd.to_datetime('2020-09-07')].copy()
df_1w = df[df['t_dat'] >= pd.to_datetime('2020-09-15')].copy()

In [ ]:
len(df_3w)

In [ ]:
df_2w.head()

In [ ]:
df_1w.head()

In [ ]:
purchase_dict_3w = {}

for i,x in enumerate(zip(df_3w['customer_id'], df_3w['article_id'])):
    cust_id, art_id = x
    if cust_id not in purchase_dict_3w:
        purchase_dict_3w[cust_id] = {}
    
    if art_id not in purchase_dict_3w[cust_id]:
        purchase_dict_3w[cust_id][art_id] = 0
    
    purchase_dict_3w[cust_id][art_id] += 1
    
print(len(purchase_dict_3w))

dummy_list_3w = list((df_3w['article_id'].value_counts()).index)[:12]

In [ ]:
purchase_dict_2w = {}

for i,x in enumerate(zip(df_2w['customer_id'], df_2w['article_id'])):
    cust_id, art_id = x
    if cust_id not in purchase_dict_2w:
        purchase_dict_2w[cust_id] = {}
    
    if art_id not in purchase_dict_2w[cust_id]:
        purchase_dict_2w[cust_id][art_id] = 0
    
    purchase_dict_2w[cust_id][art_id] += 1
    
print(len(purchase_dict_2w))

dummy_list_2w = list((df_2w['article_id'].value_counts()).index)[:12]

In [ ]:
purchase_dict_1w = {}

for i,x in enumerate(zip(df_1w['customer_id'], df_1w['article_id'])):
    cust_id, art_id = x
    if cust_id not in purchase_dict_1w:
        purchase_dict_1w[cust_id] = {}
    
    if art_id not in purchase_dict_1w[cust_id]:
        purchase_dict_1w[cust_id][art_id] = 0
    
    purchase_dict_1w[cust_id][art_id] += 1
    
print(len(purchase_dict_1w))

dummy_list_1w = list((df_1w['article_id'].value_counts()).index)[:12]

In [ ]:
print(submission.shape)
submission.head()

In [ ]:
not_so_fancy_but_fast_benchmark = submission[['customer_id']]
prediction_list = []

dummy_list = list((df_2w['article_id'].value_counts()).index)[:12]
dummy_pred = ' '.join(dummy_list)

for i, cust_id in enumerate(submission['customer_id'].values.reshape((-1,))):
    if cust_id in purchase_dict_1w:
        l = sorted((purchase_dict_1w[cust_id]).items(), key=lambda x: x[1], reverse=True)
        l = [y[0] for y in l]
        if len(l)>12:
            s = ' '.join(l[:12])
        else:
            s = ' '.join(l+dummy_list_1w[:(12-len(l))])
    elif cust_id in purchase_dict_2w:
        l = sorted((purchase_dict_2w[cust_id]).items(), key=lambda x: x[1], reverse=True)
        l = [y[0] for y in l]
        if len(l)>12:
            s = ' '.join(l[:12])
        else:
            s = ' '.join(l+dummy_list_2w[:(12-len(l))])
    elif cust_id in purchase_dict_3w:
        l = sorted((purchase_dict_3w[cust_id]).items(), key=lambda x: x[1], reverse=True)
        l = [y[0] for y in l]
        if len(l)>12:
            s = ' '.join(l[:12])
        else:
            s = ' '.join(l+dummy_list_3w[:(12-len(l))])
    else:
        s = dummy_pred
    prediction_list.append(s)

not_so_fancy_but_fast_benchmark['prediction'] = prediction_list
print(not_so_fancy_but_fast_benchmark.shape)
not_so_fancy_but_fast_benchmark.head()

In [ ]:
not_so_fancy_but_fast_benchmark.to_csv('submission.csv', index=False)